# Benchmark 1

We study the Hamiltonian of the Heisenberg model with periodic boundary conditions.

In [1]:
using CondensedMatterSOS
import MultivariatePolynomials as MP
@spin σ[1:3]
heisenberg_hamiltonian(σ, true)

# Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

import Clarabel
solver = Clarabel.Optimizer

Clarabel.MOIwrapper.Optimizer

We can compute a lower bound `-2√2` to the ground state energy as follow:

In [2]:
function hamiltonian_energy(N, maxdegree, solver; symmetry=true, consecutive=false, kws...)
    @spin σ[1:N]
    H = heisenberg_hamiltonian(σ, true)
    G = Lattice1Group(N)
    cone = NonnegPolyInnerCone{MOI.HermitianPositiveSemidefiniteConeTriangle}()
    @assert iseven(maxdegree)
    cert = SumOfSquares.Certificate.FixedBasis(
        cone,
        MonomialBasis(MP.monomials(σ[1], 0:div(maxdegree, 2), consecutive=consecutive)),
    )
    certificate = Symmetry.Ideal(
        Symmetry.Pattern(G, Action(σ)),
        cert,
    )
    if symmetry
        energy(H, maxdegree, solver; certificate = certificate, kws...)
    else
        energy(H, maxdegree, solver; kws...)
    end
end
bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
    symmetry = false,
    sparsity = SumOfSquares.Sparsity.NoPattern(),
)
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 50
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 127
  cones (total) = 2
    : Zero        = 1,  numel = 16
    : PSDTriangle = 1,  numel = 105

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       g

-5.999999993853594

We can see that the moment matrix uses all monomials:

In [3]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σᶻ₂
 σʸ₂
 σˣ₂
 σᶻ₁
 σʸ₁
 σˣ₁

# Symmetry reduction

We can reduce the computation using symmetry reduction as follows.

In [4]:
using CondensedMatterSOS

bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
)
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 4
  constraints   = 13
  nnz(P)        = 0
  nnz(A)        = 16
  cones (total) = 4
    : Zero        = 1,  numel = 4
    : PSDTriangle = 3,  numel = (3,3,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       g

-5.999999997057931

The reduction is obtained by block diagonalizing with a change of polynomial
basis to the isotypical basis.

In [5]:
display([M.basis.polynomials for M in ν.blocks])

[M.basis.polynomials for M in ν.blocks]

7-element Vector{Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}}:
 [(-0.7071067811865472 + 0.0im)σᶻ₂ + (-0.7071067811865475 + 0.0im)σᶻ₁]
 [(-0.7071067811865472 + 0.0im)σʸ₂ + (-0.7071067811865475 + 0.0im)σʸ₁]
 [(-0.7071067811865472 + 0.0im)σˣ₂ + (-0.7071067811865475 + 0.0im)σˣ₁]
 [(1.0 - 0.0im)]
 [(-0.7071067811865472 + 0.0im)σᶻ₂ + (0.7071067811865475 + 0.0im)σᶻ₁]
 [(-0.7071067811865472 + 0.0im)σʸ₂ + (0.7071067811865475 + 0.0im)σʸ₁]
 [(-0.7071067811865472 + 0.0im)σˣ₂ + (0.7071067811865475 + 0.0im)σˣ₁]

7-element Vector{Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}}:
 [(-0.7071067811865472 + 0.0im)σᶻ₂ + (-0.7071067811865475 + 0.0im)σᶻ₁]
 [(-0.7071067811865472 + 0.0im)σʸ₂ + (-0.7071067811865475 + 0.0im)σʸ₁]
 [(-0.7071067811865472 + 0.0im)σˣ₂ + (-0.7071067811865475 + 0.0im)σˣ₁]
 [(1.0 - 0.0im)]
 [(-0.7071067811865472 + 0.0im)σᶻ₂ + (0.7071067811865475 + 0.0im)σᶻ₁]
 [(-0.7071067811865472 + 0.0im)σʸ₂ + (0.7071067811865475 + 0.0im)σʸ₁]
 [(-0.7071067811865472 + 0.0im)σˣ₂ + (0.7071067811865475 + 0.0im)σˣ₁]

Let's try this for 3 sites. First without symmetry.

In [6]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
    symmetry = false,
)
@show bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 29
  constraints   = 76
  nnz(P)        = 0
  nnz(A)        = 76
  cones (total) = 5
    : Zero        = 1,  numel = 10
    : PSDTriangle = 4,  numel = (3,21,21,21)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost 

-4.49999999647938

Now with symmetry.

In [7]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
)
@show bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 5
  constraints   = 22
  nnz(P)        = 0
  nnz(A)        = 40
  cones (total) = 5
    : Zero        = 1,  numel = 10
    : PSDTriangle = 4,  numel = (3,3,3,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost     

-4.499999996767557

Let's look at the isotypical basis.

In [8]:
display([M.basis.polynomials for M in ν.blocks])

10-element Vector{Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}}:
 [(-0.5773502691896257 + 0.0im)σᶻ₃ + (-0.5773502691896257 + 0.0im)σᶻ₂ + (-0.5773502691896257 + 0.0im)σᶻ₁]
 [(-0.5773502691896257 + 0.0im)σʸ₃ + (-0.5773502691896257 + 0.0im)σʸ₂ + (-0.5773502691896257 + 0.0im)σʸ₁]
 [(-0.5773502691896257 + 0.0im)σˣ₃ + (-0.5773502691896257 + 0.0im)σˣ₂ + (-0.5773502691896257 + 0.0im)σˣ₁]
 [(1.0 - 0.0im)]
 [(-0.577350269189626 + 0.0im)σᶻ₃ + (0.2886751345948131 - 0.49999999999999994im)σᶻ₂ + (0.28867513459481275 + 0.5000000000000001im)σᶻ₁]
 [(-0.577350269189626 + 0.0im)σʸ₃ + (0.2886751345948131 - 0.49999999999999994im)σʸ₂ + (0.28867513459481275 + 0.5000000000000001im)σʸ₁]
 [(-0.577350269189626 + 0.0im)σˣ₃ + (0.2886751345948131 - 0.49999999999999994im)σˣ₂ + (0.28867513459481275 + 0.5000000000000001im)σˣ₁]
 [(-0.577350269189626 + 0.0im)σᶻ₃ + (0.28867513459481275 + 0.5000000000000001im)σᶻ₂ + (0.288675134594813

Now let's define a function for our common use case.

In [9]:
function f(L, d=1, consecutive=false; symmetry=true)
    @show L
    println("***")
    @show d
    bound, gram, ν = @time hamiltonian_energy(
        L,
        2d,
        solver,
        consecutive=consecutive,
        symmetry=symmetry,
    )
    @show bound
    for M in ν.blocks
        display(round.(M.basis.polynomials, digits=6))
    end
    println("E/N = ", bound / L)
    println("------------------------------------")
end

f(6, 1, true)

L = 6
***
d = 1
-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 8
  constraints   = 67
  nnz(P)        = 0
  nnz(A)        = 292
  cones (total) = 8
    : Zero        = 1,  numel = 46
    : PSDTriangle = 7,  numel = (3,3,3,3,...,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σᶻ₆ + (-0.408248 + 0.0im)σᶻ₅ + (-0.408248 + 0.0im)σᶻ₄ + (-0.408248 + 0.0im)σᶻ₃ + (-0.408248 + 0.0im)σᶻ₂ + (-0.408248 + 0.0im)σᶻ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σʸ₆ + (-0.408248 + 0.0im)σʸ₅ + (-0.408248 + 0.0im)σʸ₄ + (-0.408248 + 0.0im)σʸ₃ + (-0.408248 + 0.0im)σʸ₂ + (-0.408248 + 0.0im)σʸ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σˣ₆ + (-0.408248 + 0.0im)σˣ₅ + (-0.408248 + 0.0im)σˣ₄ + (-0.408248 + 0.0im)σˣ₃ + (-0.408248 + 0.0im)σˣ₂ + (-0.408248 + 0.0im)σˣ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (1.0 - 0.0im)

...

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σʸ₆ + (-0.204124 + 0.353553im)σʸ₅ + (0.204124 + 0.353553im)σʸ₄ + (0.408248 + 0.0im)σʸ₃ + (0.204124 - 0.353553im)σʸ₂ + (-0.204124 - 0.353553im)σʸ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σˣ₆ + (-0.204124 + 0.353553im)σˣ₅ + (0.204124 + 0.353553im)σˣ₄ + (0.408248 + 0.0im)σˣ₃ + (0.204124 - 0.353553im)σˣ₂ + (-0.204124 - 0.353553im)σˣ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σᶻ₆ + (0.408248 + 0.0im)σᶻ₅ + (-0.408248 + 0.0im)σᶻ₄ + (0.408248 + 0.0im)σᶻ₃ + (-0.408248 + 0.0im)σᶻ₂ + (0.408248 + 0.0im)σᶻ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σʸ₆ + (0.408248 + 0.0im)σʸ₅ + (-0.408248 + 0.0im)σʸ₄ + (0.408248 + 0.0im)σʸ₃ + (-0.408248 + 0.0im)σʸ₂ + (0.408248 + 0.0im)σʸ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.408248 + 0.0im)σˣ₆ + (0.408248 + 0.0im)σˣ₅ + (-0.408248 + 0.0im)σˣ₄ + (0.408248 + 0.0im)σˣ₃ + (-0.408248 + 0.0im)σˣ₂ + (0.408248 + 0.0im)σˣ₁

Now with `d = 2`.

In [10]:
f(6, 2, true)

L = 6
***
d = 2
-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 46
  constraints   = 894
  nnz(P)        = 0
  nnz(A)        = 10021
  cones (total) = 25
    : Zero        = 1,  numel = 773
    : PSDTriangle = 24,  numel = (3,10,10,3,...,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10


1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.333333 + 0.0im)σᶻ₅σᶻ₆ + (0.166667 + 0.0im)σʸ₅σʸ₆ + (0.166667 + 0.0im)σˣ₅σˣ₆ + (-0.333333 + 0.0im)σᶻ₄σᶻ₅ + (0.166667 + 0.0im)σʸ₄σʸ₅ + (0.166667 + 0.0im)σˣ₄σˣ₅ + (-0.333333 + 0.0im)σᶻ₃σᶻ₄ + (0.166667 + 0.0im)σʸ₃σʸ₄ + (0.166667 + 0.0im)σˣ₃σˣ₄ + (-0.333333 + 0.0im)σᶻ₂σᶻ₃ + (0.166667 + 0.0im)σʸ₂σʸ₃ + (0.166667 + 0.0im)σˣ₂σˣ₃ + (-0.333333 + 0.0im)σᶻ₁σᶻ₆ + (-0.333333 + 0.0im)σᶻ₁σᶻ₂ + (0.166667 + 0.0im)σʸ₁σʸ₆ + (0.166667 + 0.0im)σʸ₁σʸ₂ + (0.166667 + 0.0im)σˣ₁σˣ₆ + (0.166667 + 0.0im)σˣ₁σˣ₂

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σʸ₅σʸ₆ + (0.288675 + 0.0im)σˣ₅σˣ₆ + (-0.288675 + 0.0im)σʸ₄σʸ₅ + (0.288675 + 0.0im)σˣ₄σˣ₅ + (-0.288675 + 0.0im)σʸ₃σʸ₄ + (0.288675 + 0.0im)σˣ₃σˣ₄ + (-0.288675 + 0.0im)σʸ₂σʸ₃ + (0.288675 + 0.0im)σˣ₂σˣ₃ + (-0.288675 + 0.0im)σʸ₁σʸ₆ + (-0.288675 + 0.0im)σʸ₁σʸ₂ + (0.288675 + 0.0im)σˣ₁σˣ₆ + (0.288675 + 0.0im)σˣ₁σˣ₂

2-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σᶻ₅σʸ₆ + (-0.288675 + 0.0im)σʸ₅σᶻ₆ + (-0.288675 + 0.0im)σᶻ₄σʸ₅ + (-0.288675 + 0.0im)σʸ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₃σʸ₄ + (-0.288675 + 0.0im)σʸ₃σᶻ₄ + (-0.288675 + 0.0im)σᶻ₂σʸ₃ + (-0.288675 + 0.0im)σʸ₂σᶻ₃ + (-0.288675 + 0.0im)σᶻ₁σʸ₆ + (-0.288675 + 0.0im)σᶻ₁σʸ₂ + (-0.288675 + 0.0im)σʸ₁σᶻ₆ + (-0.288675 + 0.0im)σʸ₁σᶻ₂
 (0.38257 + 0.142501im)σˣ₆ + (0.38257 + 0.142501im)σˣ₅ + (0.38257 + 0.142501im)σˣ₄ + (0.38257 + 0.142501im)σˣ₃ + (0.38257 + 0.142501im)σˣ₂ + (0.38257 + 0.142501im)σˣ₁

2-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σᶻ₅σˣ₆ + (-0.288675 + 0.0im)σˣ₅σᶻ₆ + (-0.288675 + 0.0im)σᶻ₄σˣ₅ + (-0.288675 + 0.0im)σˣ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₃σˣ₄ + (-0.288675 + 0.0im)σˣ₃σᶻ₄ + (-0.288675 + 0.0im)σᶻ₂σˣ₃ + (-0.288675 + 0.0im)σˣ₂σᶻ₃ + (-0.288675 + 0.0im)σᶻ₁σˣ₆ + (-0.288675 + 0.0im)σᶻ₁σˣ₂ + (-0.288675 + 0.0im)σˣ₁σᶻ₆ + (-0.288675 + 0.0im)σˣ₁σᶻ₂
 (0.38257 + 0.142501im)σʸ₆ + (0.38257 + 0.142501im)σʸ₅ + (0.38257 + 0.142501im)σʸ₄ + (0.38257 + 0.142501im)σʸ₃ + (0.38257 + 0.142501im)σʸ₂ + (0.38257 + 0.142501im)σʸ₁

...

2-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σʸ₅σˣ₆ + (-0.288675 + 0.0im)σˣ₅σʸ₆ + (0.288675 + 0.0im)σʸ₄σˣ₅ + (0.288675 + 0.0im)σˣ₄σʸ₅ + (-0.288675 + 0.0im)σʸ₃σˣ₄ + (-0.288675 + 0.0im)σˣ₃σʸ₄ + (0.288675 + 0.0im)σʸ₂σˣ₃ + (0.288675 + 0.0im)σˣ₂σʸ₃ + (0.288675 + 0.0im)σʸ₁σˣ₆ + (-0.288675 + 0.0im)σʸ₁σˣ₂ + (0.288675 + 0.0im)σˣ₁σʸ₆ + (-0.288675 + 0.0im)σˣ₁σʸ₂
 (0.347002 - 0.215073im)σᶻ₆ + (-0.347002 + 0.215073im)σᶻ₅ + (0.347002 - 0.215073im)σᶻ₄ + (-0.347002 + 0.215073im)σᶻ₃ + (0.347002 - 0.215073im)σᶻ₂ + (-0.347002 + 0.215073im)σᶻ₁

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.235702 - 0.0im)σᶻ₅σᶻ₆ + (-0.235702 - 0.0im)σʸ₅σʸ₆ + (-0.235702 - 0.0im)σˣ₅σˣ₆ + (0.235702 - 0.0im)σᶻ₄σᶻ₅ + (0.235702 - 0.0im)σʸ₄σʸ₅ + (0.235702 - 0.0im)σˣ₄σˣ₅ + (-0.235702 - 0.0im)σᶻ₃σᶻ₄ + (-0.235702 - 0.0im)σʸ₃σʸ₄ + (-0.235702 - 0.0im)σˣ₃σˣ₄ + (0.235702 - 0.0im)σᶻ₂σᶻ₃ + (0.235702 - 0.0im)σʸ₂σʸ₃ + (0.235702 - 0.0im)σˣ₂σˣ₃ + (0.235702 - 0.0im)σᶻ₁σᶻ₆ + (-0.235702 - 0.0im)σᶻ₁σᶻ₂ + (0.235702 - 0.0im)σʸ₁σʸ₆ + (-0.235702 - 0.0im)σʸ₁σʸ₂ + (0.235702 - 0.0im)σˣ₁σˣ₆ + (-0.235702 - 0.0im)σˣ₁σˣ₂

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σᶻ₅σʸ₆ + (0.288675 + 0.0im)σʸ₅σᶻ₆ + (0.288675 + 0.0im)σᶻ₄σʸ₅ + (-0.288675 + 0.0im)σʸ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₃σʸ₄ + (0.288675 + 0.0im)σʸ₃σᶻ₄ + (0.288675 + 0.0im)σᶻ₂σʸ₃ + (-0.288675 + 0.0im)σʸ₂σᶻ₃ + (-0.288675 + 0.0im)σᶻ₁σʸ₆ + (-0.288675 + 0.0im)σᶻ₁σʸ₂ + (0.288675 + 0.0im)σʸ₁σᶻ₆ + (0.288675 + 0.0im)σʸ₁σᶻ₂

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σᶻ₅σˣ₆ + (0.288675 + 0.0im)σˣ₅σᶻ₆ + (0.288675 + 0.0im)σᶻ₄σˣ₅ + (-0.288675 + 0.0im)σˣ₄σᶻ₅ + (-0.288675 + 0.0im)σᶻ₃σˣ₄ + (0.288675 + 0.0im)σˣ₃σᶻ₄ + (0.288675 + 0.0im)σᶻ₂σˣ₃ + (-0.288675 + 0.0im)σˣ₂σᶻ₃ + (-0.288675 + 0.0im)σᶻ₁σˣ₆ + (-0.288675 + 0.0im)σᶻ₁σˣ₂ + (0.288675 + 0.0im)σˣ₁σᶻ₆ + (0.288675 + 0.0im)σˣ₁σᶻ₂

1-element Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}:
 (-0.288675 + 0.0im)σʸ₅σˣ₆ + (0.288675 + 0.0im)σˣ₅σʸ₆ + (0.288675 + 0.0im)σʸ₄σˣ₅ + (-0.288675 + 0.0im)σˣ₄σʸ₅ + (-0.288675 + 0.0im)σʸ₃σˣ₄ + (0.288675 + 0.0im)σˣ₃σʸ₄ + (0.288675 + 0.0im)σʸ₂σˣ₃ + (-0.288675 + 0.0im)σˣ₂σʸ₃ + (-0.288675 + 0.0im)σʸ₁σˣ₆ + (-0.288675 + 0.0im)σʸ₁σˣ₂ + (0.288675 + 0.0im)σˣ₁σʸ₆ + (0.288675 + 0.0im)σˣ₁σʸ₂

| id     | irep 1 | irep 2 | irep 3 | irep 4 |
|--------|--------|--------|--------|--------|
| degree | 2      | 3      | 1      | 3      |
| mult 2 | 1      | 3      | 2      | 1      |
| mult 3 | 3      | 6      | 4      | 3      |
| mult 4 | 6      | 10     | 7      | 6      |
| mult 5 | 10     | 15     | 11     | 10     |
| mult 6 | 15     | 21     | 16     | 15     |
| mult 7 | 21     | 28     | 22     | 21     |

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*